# Generación del dataset roar_rating

Tomando como base el dataset [Yahoo Answers Topics](https://huggingface.co/datasets/yahoo_answers_topics), se ha generado un nuevo dataset. Donde se tienen un cantidad de usuarios que valoran algunas de las entradas del dataset antes mencionado.

El dataset contiene 1400000 entradas, y sigue la estructura descrita a continuación:

- Categoría
- Titulo de la pregunta
- Cuerpo de la pregunta
- Mejor respuesta

Las categorías en el dataset son las siguientes:

1. Cultura y sociedad
2. Matemática y ciencias
3. Salud
4. Educación
5. Internet y computación
6. Deportes
7. Finanzas y negocios
8. Entretenimiento
9. Familia
10. Gobiernos y política

### Grafo de probabilidades de las preferencias

El grafo de probabilidades de las preferencias  es un grafo ponderado donde los pesos $p_{i, j}$ están en el intervalo $[0,1]$ y representan las probabildades de que si a un usuario $U$ le gusta la categoría $i$ le guste la $j$

In [1]:
from numpy import array, zeros
from random import randint

graph = array([
    [1.0, 0.3, 0.4, 0.6, 0.2, 0.3, 0.5, 0.6, 0.8, 0.9],
    [0.1, 1.0, 0.2, 0.7, 0.8, 0.5, 0.7, 0.65, 0.5, 0.8],
    [0.78, 0.1, 1.0, 0.6, 0.2, 0.5, 0.2, 0.7, 0.65, 0.79],
    [0.6, 0.4, 0.2, 1.0, 0.2, 0.45, 0.3, 0.5, 0.8, 0.7],
    [0.18, 0.86, 0.4, 0.3, 1.0, 0.67, 0.56, 0.7, 0.4, 0.69],
    [0.2, 0.3, 0.6, 0.2, 0.1, 1.0, 0.5, 0.7, 0.3, 0.4],
    [0.7, 0.6, 0.3, 0.5, 0.6, 0.58, 1.0, 0.62, 0.48, 0.53],
    [0.8, 0.1, 0.2, 0.1, 0.8, 0.82, 0.458, 1.0, 0.23, 0.38],
    [0.6, 0.1, 0.7, 0.75, 0.4, 0.5, 0.64, 0.58, 1.0, 0.4],
    [0.9, 0.2, 0.4, 0.6, 0.3, 0.2, 0.86, 0.42, 0.21, 1.0]
])


### Generar los vectores de probabilidades de los usuarios $p_{c_U}$

El vector de probabilidades de los usuarios representa la probabilidad de que al usuario $U$ le guste la categoría $c_i$.

In [2]:
max_categories_to_select = 4

def get_user(id: int):
    categories = [i for i in range(10)]
    user = zeros(10)

    count_category = randint(1,max_categories_to_select)
    for i in range(count_category):
        # select random index of categories
        index = randint(i, len(categories) -1)
        # update user preference
        user += graph[categories[index]]
        # put this index to the begining of the categories array
        category = categories.pop(index)
        categories.insert(0, category)
    
    return user / max_categories_to_select

#### Generar usuarios

Se generan 5000 usuarios en el dataset.

In [3]:
count_user = 5000

users = [get_user(i) for i in range(count_user)]

In [4]:
from pandas import DataFrame

users = DataFrame(users, columns=[i for i in range(10)])
users.insert(0, "id", [i for i in range(count_user)])
users.to_csv("./users.csv", index=False)

### Trabajo con el dataset Yahoo Answer Topics

Como el fin del script es generar un dataset de "roars" y valoraciones, vamos a tomar el título de las preguntas y la mejor respuesta y vamos a realizar resumenes automáticos de los mismos, para conseguir textos de a lo sumo 260 caracteres que es el maximo de la aplicacion Roar.

NO SE HIZO LO DE ARRIBA PQ ERA MUY LENTO 

Nos quedamos con los textos que tenian menos de 512 caracteres

In [5]:
import pandas as pd

max_character_by_roar = 512

In [6]:
dataset = pd.read_csv("./yahoo_answers_topic.csv")
dataset.columns = ["category", "question_title", "question_body", "best_answer"]

# insert id & text columns
dataset.insert(0, "id", [i+1 for i in range(len(dataset["category"]))])
dataset.insert(5, "text", dataset["question_title"] + [" " for i in range(len(dataset["question_title"]))] + dataset["best_answer"])
dataset.insert(6, "roar_text", ["" if len(str(text)) > max_character_by_roar else str(text) for text in dataset["text"]])

# remove unnecessary columns
dataset = dataset.drop(["question_title", "question_body", "best_answer", "text"], axis=1)

# remove empty roar text
dataset = dataset[dataset["roar_text"] != ""]


In [7]:
dataset.head()

,id,category,roar_text
0,1,2,Why does Zebras have stripes? this provides ca...
1,2,4,What did the itsy bitsy sipder climb up? water...
3,4,3,Why do women get PMS? Premenstrual syndrome (P...
4,5,3,If your co-worker is guilty of unsanitary hygi...
6,7,3,What are the risks of alternative medicine? Th...


In [8]:
dataset.shape

(46769, 3)

In [9]:
dataset.to_csv("./roars.csv", index=False)

### Generar ratings

Por cada usuario se hace lo siguiente:

- Se calcula el promedio de gusto de cada usuario, y se agrupan los post por categorias

In [103]:
from numpy.random import uniform, sample
from heapq import nlargest
import numpy as np

data_by_category = dataset.groupby("category")
groups = [data_by_category.get_group(i)["id"] for i in range(1, 11)]
len_data_by_category = [len(data_by_category.get_group(i)) for i in range(1, 11)]

users = pd.read_csv("./users.csv")
users.drop(["id"], axis=1, inplace=True)
users = users.to_numpy()

means = [np.mean(users[i]) for i in range(count_user)]

Un rating es dado un usuario y una categoria, un valor [-10, 10], aunque en la aplicación va de menos infinito a infinito. Se genera un random uniforme de [0, 10] y si el gusto de dicho usuario por la categoria es menor que el promedio entonces devuelve el rating negativo y si no el rating positivo

In [104]:
def get_rating(category: int, i: int):
    r = round(uniform(0,10), 2)
    if users[i][category] < means[i]:
        return -r
    return r


Vamos por cada usuario tomando a lo sumo las 3 categorias que mas le gustan, y calculamos cuantos post van a sufrir la interaccion del usuario, probabilidad de que le guste la categoria * la cantidad de post en la categoria y un uniforme (0, 0.8). Luego se escogen esa cantidad de post al azar y se aplica la funcion get_rating.

In [105]:
user_road = []
for u in range(count_user):
    n = randint(1, 3)
    heap = nlargest(n, list(zip(users[u], range(1, 11))), key=lambda x: x[0])
    for i in range(n):
        p, c = heap[i]
        count = int(p * len_data_by_category[c-1] * uniform(0, 0.8))
        
        smp =  list(groups[c-1].sample(count))
        for s in smp:
            user_road.append([u, s, get_rating(c-1, u)]) 


In [106]:
user_ratings = DataFrame(user_road, columns=["user_id", "roar_id", "rating"])

In [107]:
user_ratings = user_ratings.sort_values(by=["user_id", "roar_id"])
user_ratings.head()

,user_id,roar_id,rating
899,0,2,9.59
397,0,18,6.07
240,0,25,5.25
315,0,55,4.40
158,0,114,1.60


In [108]:
user_ratings.to_csv("./user_ratings.csv", index=False)